In [1]:
pip install spacy


In [2]:
pip install -U benepar


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00
  Created wheel for benepar: filename=benepar-0.2.0-py3-none-any.whl size=37625 sha256=ca787be24586d6b9f7e0b6bb718d48138ee5057f6e66baad87803d18065b5ba8
  Stored i

In [3]:
pip install datasets


In [4]:
import os
import spacy
from collections import Counter
# import shifterator as sh
import matplotlib.pyplot as plt
import benepar as bnp
from benepar.spacy_plugin import BeneparComponent
from nltk.tree import Tree
from nltk.tokenize import sent_tokenize
from collections import defaultdict
import nltk
from datasets import load_from_disk




/usr/local/lib/python3.11/dist-packages/benepar/spacy_plugin.py:7: FutureWarning: BeneparComponent and NonConstituentException have been moved to the benepar module. Use `from benepar import BeneparComponent, NonConstituentException` instead of benepar.spacy_plugin. The benepar.spacy_plugin namespace is deprecated and will be removed in a future version.
  warnings.warn(


In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# benepar
import benepar
benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.


True

In [8]:
# spacy model load
nlp = spacy.load("en_core_web_sm")

# benepar constituency parser addition (ERRORS SOMETIMES!!)
nlp.add_pipe("benepar", config = {'model': 'benepar_en3'})


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
pre_folder = '/content/drive/MyDrive/TFG/PRE_ELEC_SCRIPTS'
pre_files = [os.path.join(pre_folder, f) for f in os.listdir(pre_folder) if f.endswith('.txt')]

post_folder = '/content/drive/MyDrive/TFG/POST_ELEC_SCRIPTS'
post_files = [os.path.join(post_folder, f) for f in os.listdir(post_folder) if f.endswith('.txt')]

## Constituency parsing (tree)

In [ ]:
keywords = {"my administration", "immigrants", "islam", "hillary clinton"}


In [ ]:

# New function to extract sentences from a HuggingFace dataset
def read_sentences_from_dataset(dataset, text_column="sentence"):
    all_sentences = []
    for row in dataset:
        text = row[text_column].lower()
        all_sentences.append(text)
    return all_sentences


# Reused function (no changes needed)
def collect_keyword_sentences(sentences, keywords, max_sentences=100, max_char_len=500):
    keyword_sents = defaultdict(list)

    for sent in sentences:
        sent = sent.strip()
        if len(sent) > max_char_len:
            continue
        for keyword in keywords:
            if keyword in sent:
                keyword_sents[keyword].append(sent)

    for k in keyword_sents:
        sents = sorted(keyword_sents[k], key=len, reverse=True)
        keyword_sents[k] = sents[:max_sentences]

    return keyword_sents


In [ ]:
# Find adjectives within the same noun phrase as a keyword
def find_adjectives_in_tree(tree, keyword):
    adjs = []

    for subtree in tree.subtrees(lambda t: t.label() == "NP"):
        pos_tags = subtree.pos()
        phrase_text = " ".join(w for w, _ in pos_tags).lower()

        if keyword in phrase_text:
            for word, tag in pos_tags:
                if tag.startswith("JJ") or tag in ("VBG", "VBN"):
                    adjs.append(word.lower())

    return adjs

# Apply to all sentences
def extract_adjectives_for_keywords(sentences_by_keyword):
    results = defaultdict(list)

    for keyword, sentences in sentences_by_keyword.items():
        for sentence in sentences:
            try:
                doc = nlp(sentence)
                for sent in doc.sents:
                    tree = Tree.fromstring(sent._.parse_string)
                    adjs = find_adjectives_in_tree(tree, keyword)
                    results[keyword].extend(adjs)
            except Exception as e:
                print(f"Skipping sentence due to error: {e}")
                continue

    # deduplication, can be skipped to get adjective frequency if needed
    for k in results:
        results[k] = list(set(results[k]))

    return results



def get_example_sentences_per_keyword(sentences_by_keyword, nlp, max_examples=15):
    examples = defaultdict(list)

    for keyword, sentences in sentences_by_keyword.items():
        count = 0
        for sentence in sentences:
            if count >= max_examples:
                break
            try:
                doc = nlp(sentence)
                for sent in doc.sents:
                    tree = Tree.fromstring(sent._.parse_string)
                    # Search for NP containing the keyword
                    for subtree in tree.subtrees(lambda t: t.label() == "NP"):
                        phrase_text = " ".join(w for w, _ in subtree.pos()).lower()
                        if keyword in phrase_text:
                            examples[keyword].append(sentence)
                            count += 1
                            break  # Move to next sentence after match
            except Exception as e:
                print(f"Skipping sentence due to error: {e}")
                continue

    return examples

In [ ]:
# Load datasets
pre_dataset = load_from_disk("/content/drive/MyDrive/TFG/sent_datasets/pre_dataset_cache")
post_dataset = load_from_disk("/content/drive/MyDrive/TFG/sent_datasets/post_dataset_cache")


pre_sentences = read_sentences_from_dataset(pre_dataset)
post_sentences = read_sentences_from_dataset(post_dataset)

In [ ]:
# keyword sentence extraction
pre_results = collect_keyword_sentences(pre_sentences, keywords)
post_results = collect_keyword_sentences(post_sentences, keywords)

In [ ]:
# Adjective extraction
pre_adjs = extract_adjectives_for_keywords(pre_results)
post_adjs = extract_adjectives_for_keywords(post_results)



/usr/local/lib/python3.11/dist-packages/torch/distributions/distribution.py:56: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


In [ ]:
#Exemplary sentences
sents_pre = get_example_sentences_per_keyword(pre_results, nlp)
sents_post = get_example_sentences_per_keyword(post_results, nlp)

In [ ]:
len(pre_adjs['immigrants'])

79

In [ ]:
post_adjs['immigrants']

['illegal',
 'united',
 'tremendous',
 'going',
 'made',
 'non',
 'irish',
 'multiple']

In [ ]:
sents_post['immigrants']

['we had one instance in australia — i have a lot of respect for australia, i love australia as a country — but we had a problem where, for whatever reason, president obama said that they were going to take probably well over a thousand illegal immigrants who were in prisons and they were going to bring them and take them into this country.',
 'we’re here today to celebrate america’s commitment to ireland and the tremendous contributions — and i know it well — the irish immigrants and their descendants have made right here in the united states and throughout the world.',
 'they’ve had members of their family killed by illegal immigrants and, really, people with multiple — in some cases, multiple deportations.',
 'suspend the entry of all aliens or any class of aliens as immigrants or non-immigrants, or impose on the entry of aliens.']

## Extras

In [ ]:
# not necessary if add_pipe above works properly

# if not nlp.has_pipe("benepar"):
#     try:
#         nlp.add_pipe("benepar", config={"model": "benepar_en2"})
#     except Exception as e:
#         print(f"Error loading benepar model: {e}")

In [ ]:
from nltk.tree import Tree
import traceback

# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("benepar", config={"model": "benepar_en3"})

def find_adjectives_in_tree(tree, keywords):
    keyword_adjs = defaultdict(list)

    for subtree in tree.subtrees(lambda t: t.label() == "NP"):
        pos_tags = subtree.pos()

        # Build noun phrase and adjective list
        phrase_words = [w for w, _ in pos_tags]
        phrase = " ".join(phrase_words).lower()

        adjs = [w for w, t in pos_tags if t.startswith("JJ") or t in ("VBN", "VBG")]

        # Match full phrase or any part of it
        for keyword in keywords:
            if keyword in phrase:
                keyword_adjs[keyword].extend([adj.lower() for adj in adjs])

    return keyword_adjs


def parse_and_extract_adjs(doc, keywords):
    results = defaultdict(list)
    for s in doc.sents:
        try:
            tree = Tree.fromstring(s._.parse_string)
            adjs = find_adjectives_in_tree(tree, keywords)
            for k, v in adjs.items():
                results[k].extend(v)
        except Exception as e:
            print(f"Skipping sentence due to error: {e}")
    return results


def process_all_files(file_list, keywords):
    all_results = defaultdict(list)

    for file_path in file_list:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read().lower()
            try:
                doc = nlp(text)
            except Exception as e:
                print(f"\n❌ Error in file: {file_path}")
                traceback.print_exc()
                continue

            file_results = parse_and_extract_adjs(doc, keywords)

            for k, v in file_results.items():
                all_results[k].extend(v)

    return all_results




In [ ]:
keywords = {"united states", "hillary clinton", "america", "our country", "my opponent", "barack obama", "our campaign", "donald trump", "your jobs", "our jobs", "illegal immigration", "immigration"}  # customize as needed

results = process_all_files(pre_files, keywords)

for k in results:
    results[k] = list(set(results[k]))

# output
print(results)


❌ Error in file: /content/drive/MyDrive/TFG/PRE_ELEC_SCRIPTS/Remarks_Announcing_Candidacy_for_President_in_New_York_City.txt

❌ Error in file: /content/drive/MyDrive/TFG/PRE_ELEC_SCRIPTS/Statement_by_Donald_J__Trump_on_Veterans_Day.txt


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/benepar/retokenization.py", line 74, in retokenize
    token_idx, (token_start, token_end) = next(offset_mapping_iter)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-26-11fd102db1c0>", line 48, in process_all_files
    doc = nlp(text)
          ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1057, in __call__
    error_handler(name, proc, [doc], e)
  File "/usr/local/lib/python3.11/dist-packages/spacy/util.py", line 1722, in raise_error
    raise e
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1052, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))  # type: ignore[call-arg]
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages


❌ Error in file: /content/drive/MyDrive/TFG/PRE_ELEC_SCRIPTS/Statement_by_Donald_J__Trump_Responding_to_the_Lies_of_Senator_Cruz_and_Warns_of_Legal_Action.txt


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/benepar/retokenization.py", line 74, in retokenize
    token_idx, (token_start, token_end) = next(offset_mapping_iter)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-26-11fd102db1c0>", line 48, in process_all_files
    doc = nlp(text)
          ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1057, in __call__
    error_handler(name, proc, [doc], e)
  File "/usr/local/lib/python3.11/dist-packages/spacy/util.py", line 1722, in raise_error
    raise e
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1052, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))  # type: ignore[call-arg]
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages


❌ Error in file: /content/drive/MyDrive/TFG/PRE_ELEC_SCRIPTS/Remarks_at_the_AIPAC_Policy_Conference_in_Washington__DC.txt


...


❌ Error in file: /content/drive/MyDrive/TFG/PRE_ELEC_SCRIPTS/Remarks_Introducing_Governor_Mike_Pence_as_the_2016_Republican_Vice_Presidential_Nominee_in_New_York_City.txt


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/benepar/retokenization.py", line 74, in retokenize
    token_idx, (token_start, token_end) = next(offset_mapping_iter)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-26-11fd102db1c0>", line 48, in process_all_files
    doc = nlp(text)
          ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1057, in __call__
    error_handler(name, proc, [doc], e)
  File "/usr/local/lib/python3.11/dist-packages/spacy/util.py", line 1722, in raise_error
    raise e
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1052, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))  # type: ignore[call-arg]
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages


❌ Error in file: /content/drive/MyDrive/TFG/PRE_ELEC_SCRIPTS/Address_Accepting_the_Presidential_Nomination_at_the_Republican_National_Convention_in_Cleveland__Ohio.txt


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/benepar/retokenization.py", line 74, in retokenize
    token_idx, (token_start, token_end) = next(offset_mapping_iter)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-26-11fd102db1c0>", line 48, in process_all_files
    doc = nlp(text)
          ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1057, in __call__
    error_handler(name, proc, [doc], e)
  File "/usr/local/lib/python3.11/dist-packages/spacy/util.py", line 1722, in raise_error
    raise e
  File "/usr/local/lib/python3.11/dist-packages/spacy/language.py", line 1052, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))  # type: ignore[call-arg]
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages

KeyboardInterrupt: 

In [ ]:
# tree
def tree_parser(doc):
  for s in doc.sents:
    print(s._.parse_string)

  tree = Tree.fromstring(s._.parse_string)
  tree.pretty_print()

In [ ]:
sent = "so even though you say we want, as an example, general electric to produce more, if they don't want to or if for some reason one of the donors of crooked hillary clinton doesn't want that to happen, even though it's great for erie, even though it's great you, even though it's great for the state of pennsylvania, then it's not going to happen, folks."
sdoc = nlp(sent) #tree_nlp

tree_parser(sdoc)

(S (RB so) (SBAR (SBAR (ADVP (RB even)) (IN though) (S (NP (PRP you)) (VP (VBP say) (SBAR (S (NP (PRP we)) (VP (VBP want) (, ,) (PP (IN as) (NP (DT an) (NN example))) (, ,) (NP (NP (JJ general) (NN electric)) (SBAR (S (VP (TO to) (VP (VB produce) (NP (JJR more))))))))))))) (, ,) (SBAR (SBAR (IN if) (S (NP (PRP they)) (VP (VBP do) (RB n't) (VP (VB want) (S (VP (TO to))))))) (CC or) (SBAR (IN if) (S (PP (IN for) (NP (DT some) (NN reason))) (NP (NP (CD one)) (PP (IN of) (NP (NP (DT the) (NNS donors)) (PP (IN of) (NP (JJ crooked) (JJ hillary) (NN clinton)))))) (VP (VBZ does) (RB n't) (VP (VB want) (S (NP (DT that)) (VP (TO to) (VP (VB happen)))))))))) (, ,) (SBAR (SBAR (SBAR (ADVP (RB even)) (IN though) (S (NP (PRP it)) (VP (VBZ 's) (ADJP (JJ great)) (PP (IN for) (NP (NN erie)))))) (, ,) (SBAR (SBAR (ADVP (RB even)) (IN though) (S (NP (PRP it)) (VP (VBZ 's) (ADJP (JJ great)) (PRP you)))) (, ,) (SBAR (ADVP (RB even)) (IN though) (S (NP (PRP it)) (VP (VBZ 's) (JJ great) (PP (IN for) (NP (NP 